# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [287]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2022-04-27 19:45:34--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg.3’

La_noche_estrellada 100%[===================>] 218.48K  1.39MB/s    in 0.2s    

2022-04-27 19:45:35 (1.39 MB/s) - ‘La_noche_estrellada1.jpg.3’ saved [223725/223725]

--2022-04-27 19:45:35--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1530

In [288]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [289]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/bariloche.jpeg")
style_reference_image_path = Path("/content/929px-Starry_Night_Over_the_Rhone.webp")
result_prefix = Path("/content/output6")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta: 

*   total_variation_weight: peso utlizado en el calculo de la loss y es encargado de penalizar la varianza de la imagen generada dandole coherencia y suavidad en los cambios entre pixeles

*   style_weight (alfa): define la importancia de respetar los estilos de la imagen que aporta estilos (valga la redundancia) al momento de calcular el loss.

*   content_weight (beta): define la importancia de respetar la forma de la imagen que aporta contenido.

Entre estos dos parametros (alfa y beta) vamos a tener un trade-off entre contenido y estilo.



```
Ltotal(~p,~a, ~x) = αLcontent(~p, ~x) + βLstyle(~a, ~x) 
```






In [290]:
total_variation_weight = 0.3
style_weight = 10
content_weight = 5

In [291]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta: 

- Carga la imagen y la redimenciona para adecularla al modelo (VGG19).
- Convierte la imagen a un numpy array.
- Expande el array de la linea anterior en 1 dimension para usarlo como input de la red.
- Adecua la imagen (el array que la representa) a los requerimientos del modelo (VGG19), es decir, la convierte de RGB a BGR. Luego cada canal de color se centra en cero con respecto al dataset de ImageNet, sin escalado.

In [292]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta: Realiza el proceso inverso, descentrando la imagen y convirtiendola a RGB.

In [293]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [294]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [295]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [296]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [297]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

La matrix de Gram es el producto punto entre filtros, permite estimar la relacion entre los filtros de los diferentes layers y obtener cual es la correlacion entre ellos.
Es utilizado para calcular el Style Loss (de forma tal que se mantenga la correlacion de los estilos. EX: Circulos que se ven en la imagen C de la pagina 5 del paper).

- ¿Por qué se permutan las dimensiones de x?

De esta forma se permite encontrar aquellos features que se activan simultaneamente en una misma posicion.

In [298]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

*   style_loss: Consisten en utilizar la Gram Matrix generada entre la imagen original y la imagen generada para compararlas y determinar que tan bien nuestra red esta replicando el estilo de la imagen original, de forma tal que al minimizar esta diferencia, obtendremos una imagen cuyo estilo (texturas/colores) sea lo mas cercano a la imagen original posible.

*   content_loss: Mide la distancia L2 de los features geneados por la red y los features de la content image base, para que al minimizar esta distancia, obtenegamos una imagen cuyo contenido sea lo mas parecido a la imagen real posible.

*   total_variation_loss: Mide la transicion entre pixeles de la imagen generada, permitiendo genear una transicion suave al ser minimizado. Permite mantener la coherencia y continuidad espacial del output de la red. Genera un efecto normalizador, no es parte del paper original.



In [299]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [300]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [301]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [302]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [303]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

*   Primer celda: Funcion utilizada para la evaluacion del loss y el gadiente.

*   Segunda celda: Define una clase llamada Evaluator(), encargada de calcular el loss y los grads de forma simultanea para realizar el proceso de una forma mas eficiente (debido que el optimizado utilizado, L-BFGS, requiere que se calculen en funciones separadas).

*   Tercer celda: Loop del optimizador. Se itera n veces, 100 en este caso. Por cada loop se calcula la loss y se actualizan los pesos a partir de los gradientes, se deprocesa la imagen (punto 3 de este trabajo) y se guarda el resultado como resultado de la interacion para su posterior visualizacion.


El paper define la utilizacion del Style Loss y el Content Loss, mientras que en este trabajo se utiliza tambien el Total Variation Loss.

Un metodo alternativo, podria ser utilizar otro optimizador como SGD o Adam.

In [304]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [305]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 27832705000.0
Image saved as /content/output6/output_at_iteration_0.png
Iteration 0 completed in 20s
Start of iteration 1
Current loss value: 15226663000.0
Image saved as /content/output6/output_at_iteration_1.png
Iteration 1 completed in 18s
Start of iteration 2
Current loss value: 11657843000.0
Image saved as /content/output6/output_at_iteration_2.png
Iteration 2 completed in 18s
Start of iteration 3
Current loss value: 9536422000.0
Image saved as /content/output6/output_at_iteration_3.png
Iteration 3 completed in 18s
Start of iteration 4
Current loss value: 8426078700.0
Image saved as /content/output6/output_at_iteration_4.png
Iteration 4 completed in 18s
Start of iteration 5
Current loss value: 7595171300.0
Image saved as /content/output6/output_at_iteration_5.png
Iteration 5 completed in 18s
Start of iteration 6
Current loss value: 7087392300.0
Image saved as /content/output6/output_at_iteration_6.png
Iteration 6 completed in 18s
Start of i

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta: Se probo con la siguiente combinacion de pesos (imagenes adjuntas en carpeta con nombre "Punto 8"):

*   style_weight = 10 ; content_weight = 1 ; total_variation_weight = 0.1
*   style_weight = 1 ; content_weight = 10 ; total_variation_weight = 0.1
*   style_weight = 10 ; content_weight = 1 ; total_variation_weight = 0.8
*   style_weight = 10 ; content_weight = 10 ; total_variation_weight = 0.1
*   style_weight = 10 ; content_weight = 1 ; total_variation_weight = 5

De estos casos de prueba, podemos concluir notar algunas cosas a destacar:

* Al darle mas peso al content_weight se pierden menos detalles de la foto original (Podremos observar esto si comparamos la imagen final de la prueba 1 y la prueba 2, particularmente observando las sombras dentro del agua o los detalles en el techo de la primer casa de izquierda a derecha).

* Al darle mas peso al style_weight mantenemos unos estilos mas fieles a la imagen de estilo (Podremos observar esto comparando la imagen final de la prueba 1 y la prueba 2, particularmente en los circulos amarillentos que se visualizan en el cielo sobre la imagen 2 y que tambien se encuentran presentes en la imagen de estilo pero no en la imagen que resulta de la prueba 1).

* Al darle mas peso a total_variation_weight obtendremos una imagen donde la transicion entre los elementos de la imagen sea mas amena (Se realizo primero la prueba 3 utilizando como valor 0.8, pero al no visualizar grandes cambios se relaizo luego la prueba 5 con un valor de 5 para este weight. Aqui podemos observar cierta continuidad entre los bordes de las casas, e incluso hay una en aprticular donde se puede ver que los colores amarillo y violeta se entrelazan. Tambien la imagen parece haber ganado cierto brillo o contraste).

* Al mantener mismos valores para style_weight y content_weight obtendremos un punto medio entre la variacion de los estilos y la deformacion de la imagen (esto se puede visualizar en la prueba 4, donde ambos valores fueron seteados en 5, y el resultado final es una imagen que encuentra cierto equilibrio entre los primeros dos items destacados mas arriba. Por ejemplo, las sombras del agua estan mas visibles que en el punto dos, pero no tanto como el puto 1. Esto mismo pasa con los circulos del cielo).


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: Imagenes adjuntadas en la carpeta Punto 9